In [1]:
import pandas as pd
import numpy as np
#import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from torch.utils.data import DataLoader, Dataset
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam

In [2]:
# %capture%
# !pip install transformers
!pip install transformers[torch]
# !pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 29.5 MB/s eta 0:00:00


In [3]:
# tf.random.set_seed(13)
torch.manual_seed(13)

In [4]:
from transformers import AutoModel, BertForSequenceClassification

In [5]:
dat = pd.read_excel('/content/ReTweetsOnlyQuota.xlsx')

In [6]:
dat.head()

,ID#,Tweet,TimeFrame,ECT (1 = CT; 0 = No),Retweeted,@#ofRetweets,#ofFollowersOfTweetAuthor,Follwers (1 = < 1000; 2 = >= 1000),User,Hashtags (All include #WhiteGenocide),...,AntiMiscegenation,ThreatOfForeigner,Nostalgia,OtherAsEnemy,Melodrama,Jeremiad,Satire,Utopia,Manifesto,SecretHistory
0,640,The white genocide project is realized in Euro...,1,1,0,0.0,959.0,1.0,MarkRiser,#WhiteGenocide,...,1,1,0,1,1,0,0,0,0,0
1,642,@den2114 @RobinozNT #WhiteGenocide https://t.c...,1,1,0,0.0,422.0,1.0,WhiteGenocideOz,#WhiteGenocide,...,0,1,0,1,1,0,0,0,0,0
2,652,@DuvalMagic Diversity is a code word for #Whit...,1,1,0,0.0,281.0,1.0,Brerrabbit13,#WhiteGenocide,...,1,1,0,1,1,0,0,0,0,0
3,654,@LFC_blano @RobertoGoi1 #Kalergiplan \n#whiteg...,1,1,0,0.0,6054.0,2.0,AquAhora1,#Kalergiplan #whitegenocide,...,0,1,0,1,1,1,0,0,0,1
4,658,@RobertJohnDavi Same old same old 👽 No differe...,1,1,0,0.0,399.0,1.0,Moira_EL62,#POTUS #MAGA #WhiteGenocide,...,0,0,0,1,1,0,0,0,0,0


In [7]:
dat.columns

Index(['ID#', 'Tweet', 'TimeFrame', 'ECT (1 = CT; 0 = No)', 'Retweeted',
       '@#ofRetweets', '#ofFollowersOfTweetAuthor',
       'Follwers (1 = < 1000; 2 = >= 1000)', 'User',
       'Hashtags (All include #WhiteGenocide)',
       'SentimentofTweet (1 = Pos 2 = Neu; 3 = Neg)',
       'OnlineHate (1 = Hateful; O = Not)', 'TypeOfConspiracy (do not use)',
       'AntiInstitutionalism', 'AntiGovernment', 'FutureThreatNow',
       'AntiIntellectualiam', 'AntiSemitism', 'AntiMiscegenation',
       'ThreatOfForeigner', 'Nostalgia', 'OtherAsEnemy', 'Melodrama',
       'Jeremiad', 'Satire', 'Utopia', 'Manifesto', 'SecretHistory'],
      dtype='object')

In [8]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 28 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   ID#                                          1400 non-null   int64  
 1   Tweet                                        1400 non-null   object 
 2   TimeFrame                                    1400 non-null   int64  
 3   ECT (1 = CT; 0 = No)                         1400 non-null   int64  
 4   Retweeted                                    1400 non-null   int64  
 5   @#ofRetweets                                 1328 non-null   float64
 6   #ofFollowersOfTweetAuthor                    1398 non-null   float64
 7   Follwers (1 = < 1000; 2 = >= 1000)           1398 non-null   float64
 8   User                                         1400 non-null   object 
 9   Hashtags (All include #WhiteGenocide)        1399 non-null   object 
 10  

In [9]:
emp=[]
trp=[]
for i in range(len(dat)):
  if dat.iloc[i,13:22].to_numpy().sum()>0:
    trp.append(1)
  else:
    trp.append(0)

  if dat.iloc[i,22:].to_numpy().sum()>0:
    emp.append(1)
  else:
    emp.append(0)


In [10]:
dat['emplotment']=emp
dat['trope']=trp


In [11]:
dat.iloc[:,22:]
dat['emplotment'].value_counts()


1    1056
0     344
Name: emplotment, dtype: int64

In [12]:
dat['trope'].value_counts()

1    941
0    459
Name: trope, dtype: int64

In [13]:
dat[['emplotment','trope']].value_counts()

emplotment  trope
1           1        844
0           0        247
1           0        212
0           1         97
dtype: int64

Data preprocessing

In [14]:
dat_filtered = dat[['Tweet','emplotment','trope']]

In [15]:
dat_filtered['Tweet'].nunique()

1288

In [16]:
dups=dat_filtered['Tweet'][dat_filtered['Tweet'].duplicated()]
dups.size

112

In [17]:
dat_filtered.loc[225,'Tweet']

'someone should make a blocklist to block everyone who unironically posts in #WhiteGenocide'

In [21]:
dat_filtered[dat_filtered['Tweet']=='someone should make a blocklist to block everyone who unironically posts in #WhiteGenocide']

,Tweet,emplotment,trope
115,someone should make a blocklist to block every...,0,0
225,someone should make a blocklist to block every...,1,0


In [22]:
dat_filtered[dat_filtered['Tweet'].isin(dups.values)].sort_values('Tweet')

,Tweet,emplotment,trope
1073,"""All the civilizations we know have been creat...",1,1
1338,"""All the civilizations we know have been creat...",1,1
1082,"""Racism"" is just when a White person doesn't w...",1,1
1376,"""Racism"" is just when a White person doesn't w...",1,1
712,"""Racism"" occurs when a White person doesn't wa...",1,1
...,...,...,...
734,why is it that when no one's talking about the...,1,0
706,wtf is this? can hear #thuglets laughing & mak...,1,1
476,wtf is this? can hear #thuglets laughing & mak...,0,1
1386,“Diversity” means chasing down White people. #...,1,1


In [23]:
dat_filtered=dat_filtered.drop_duplicates(keep=False)

In [24]:
len(dat_filtered)

1291

In [25]:
dat_filtered['target'] = list(dat_filtered[['emplotment','trope']].values.tolist())

<ipython-input-25-4c9d5aff5089>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat_filtered['target'] = list(dat_filtered[['emplotment','trope']].values.tolist())


In [26]:
dat_filtered.head()
dat_filtered['target'].astype(str).value_counts()

[1, 1]    757
[0, 0]    235
[1, 0]    206
[0, 1]     93
Name: target, dtype: int64

In [27]:
x_train,x_test,y_train,y_test=train_test_split(dat_filtered['Tweet'].values.tolist(),dat_filtered['target'].values.tolist(),test_size=0.3,random_state=13,stratify=dat_filtered['target'].values.tolist())

In [28]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("digitalepidemiologylab/covid-twitter-bert-v2", do_lower_case=True)

In [ ]:
x_train

In [ ]:
# x_train_tok=tokenizer(x_train,padding=True,truncation=True,return_tensors='np')
# x_test_tok=tokenizer(x_test,padding=True,truncation=True,return_tensors='np')


In [ ]:
# x_test_tok

In [ ]:
# x_train_fin=tf.data.Dataset.from_tensor_slices((dict(x_train_tok),y_train))
# x_test_fin=tf.data.Dataset.from_tensor_slices((dict(x_test_tok),y_test))

Using baseline models

In [30]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2) #"digitalepidemiologylab/covid-twitter-bert-v2"

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model


In [32]:
# model.layers[0].trainable=False
model.bert.requires_grad=False

In [ ]:
# model.summary()
for p in model.parameters():
  print(p)

In [ ]:
# model2=tf.keras.Sequential(
#     model,
#     tf.keras.layers.Dense(2,'sigmoid')
# )

In [ ]:
# model2.summary()

In [ ]:
# x_train_fin

In [45]:
max_len=list(map(lambda x:len(x),dat_filtered['Tweet'].values.tolist()))

In [47]:
max(max_len)

751

In [54]:
input_ids = []
attention_masks = []
for text in x_train:
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512, #128
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, axis=0)
attention_masks = torch.cat(attention_masks, axis=0)
labels = torch.tensor(list(y_train), dtype=torch.float32)

# # Create TensorFlow dataset
# dataset_train = tf.data.Dataset.from_tensor_slices((input_ids, attention_masks, labels))
# dataset_train = dataset_train.shuffle(100).batch(8)

train_ds = torch.utils.data.TensorDataset(input_ids, attention_masks, labels)
train_dl = DataLoader(train_ds, batch_size=8, shuffle=True)

In [55]:
input_ids = []
attention_masks = []
for text in x_test:
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, axis=0)
attention_masks = torch.cat(attention_masks, axis=0)
labels = torch.tensor(list(y_test), dtype=torch.float32)

# # Create TensorFlow dataset
# dataset_train = tf.data.Dataset.from_tensor_slices((input_ids, attention_masks, labels))
# dataset_train = dataset_train.shuffle(100).batch(8)

test_ds = torch.utils.data.TensorDataset(input_ids, attention_masks, labels)
test_dl = DataLoader(test_ds, batch_size=8, shuffle=True)

In [56]:
class MultiLabelDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoded_dict = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoded_dict['input_ids'].squeeze(),
            'attention_mask': encoded_dict['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.float)
        }

In [59]:
train_ds = MultiLabelDataset(x_train, y_train , tokenizer, max_length=512)
test_ds = MultiLabelDataset(x_test, y_test , tokenizer, max_length=512)


In [60]:
# input_ids.shape, labels.shape, attention_masks.shape
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# dataset_train

In [61]:
from transformers import Trainer,TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    #save_steps=1000,
    #save_total_limit=2,
    logging_steps=500,
    logging_dir='./logs',
    learning_rate=1e-5,
)

In [62]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    #data_collator=data_collator,
    #tokenizer=tokenizer,
)


In [63]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=339, training_loss=0.5625789946159431, metrics={'train_runtime': 258.9123, 'train_samples_per_second': 10.463, 'train_steps_per_second': 1.309, 'total_flos': 712767848970240.0, 'train_loss': 0.5625789946159431, 'epoch': 3.0})

In [64]:
pred=trainer.predict(test_ds)

In [65]:
pred_bool=[p>0.5 for p in pred.predictions]

In [66]:
print(classification_report(y_test,pred_bool))

              precision    recall  f1-score   support

           0       0.81      0.86      0.84       289
           1       0.74      0.76      0.75       255

   micro avg       0.78      0.82      0.80       544
   macro avg       0.77      0.81      0.79       544
weighted avg       0.78      0.82      0.80       544
 samples avg       0.61      0.66      0.62       544



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# from tensorflow.keras.losses import BinaryCrossentropy

# model.compile(
#     optimizer="adam",
#     loss=BinaryCrossentropy(from_logits=False),
#     metrics=["accuracy"],
# )
# model.fit(
#     dataset_train,
#     validation_data=dataset_test,
#     epochs=2,
#     batch_size=8
# )